# Create a dataset formatted for RNN examples

## Imports & Settings

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd

In [3]:
np.random.seed(42)

In [4]:
idx = pd.IndexSlice

## Build daily dataset

In [5]:
DATA_DIR = Path('..', 'data')

In [6]:
prices = (pd.read_hdf(DATA_DIR / 'assets.h5', 'quandl/wiki/prices')
          .loc[idx['2010':'2017', :], ['adj_close', 'adj_volume']])
prices.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5213135 entries, (Timestamp('2010-01-04 00:00:00'), 'A') to (Timestamp('2017-12-29 00:00:00'), 'UFI')
Data columns (total 2 columns):
 #   Column      Dtype  
---  ------      -----  
 0   adj_close   float64
 1   adj_volume  float64
dtypes: float64(2)
memory usage: 100.1+ MB


### Select most traded stocks

In [7]:
n_dates = len(prices.index.unique('date'))
dollar_vol = (prices.adj_close.mul(prices.adj_volume)
              .unstack('ticker')
              .dropna(thresh=int(.95 * n_dates), axis=1)
              .rank(ascending=False, axis=1)
              .stack('ticker'))

In [8]:
most_traded = dollar_vol.groupby(level='ticker').mean().nsmallest(500).index

In [9]:
returns = (prices.loc[idx[:, most_traded], 'adj_close']
           .unstack('ticker')
           .pct_change()
           .sort_index(ascending=False))
returns.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2013 entries, 2017-12-29 to 2010-01-04
Columns: 500 entries, AAPL to TMK
dtypes: float64(500)
memory usage: 7.7 MB


### Stack 21-day time series

In [10]:
n = len(returns)
T = 21 # days
tcols = list(range(T))
tickers = returns.columns

In [11]:
data = pd.DataFrame()
for i in range(n-T-1):
    df = returns.iloc[i:i+T+1]
    date = df.index.max()
    data = pd.concat([data, 
                      df.reset_index(drop=True).T
                      .assign(date=date, ticker=tickers)
                      .set_index(['ticker', 'date'])])
data = data.rename(columns={0: 'label'}).sort_index().dropna()
data.loc[:, tcols[1:]] = (data.loc[:, tcols[1:]].apply(lambda x: x.clip(lower=x.quantile(.01),
                                                  upper=x.quantile(.99))))
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 995499 entries, ('A', Timestamp('2010-02-04 00:00:00')) to ('UDR', Timestamp('2017-12-29 00:00:00'))
Data columns (total 22 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   label   995499 non-null  float64
 1   1       995499 non-null  float64
 2   2       995499 non-null  float64
 3   3       995499 non-null  float64
 4   4       995499 non-null  float64
 5   5       995499 non-null  float64
 6   6       995499 non-null  float64
 7   7       995499 non-null  float64
 8   8       995499 non-null  float64
 9   9       995499 non-null  float64
 10  10      995499 non-null  float64
 11  11      995499 non-null  float64
 12  12      995499 non-null  float64
 13  13      995499 non-null  float64
 14  14      995499 non-null  float64
 15  15      995499 non-null  float64
 16  16      995499 non-null  float64
 17  17      995499 non-null  float64
 18  18      995499 non-null  float64
 19  19      99549

In [12]:
data.shape

(995499, 22)

In [13]:
data.to_hdf('data.h5', 'returns_daily')

## Build weekly dataset

We load the Quandl adjusted stock price data:

In [14]:
prices = (pd.read_hdf(DATA_DIR / 'assets.h5', 'quandl/wiki/prices')
          .adj_close
          .unstack().loc['2007':])
prices.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2896 entries, 2007-01-01 to 2018-03-27
Columns: 2925 entries, A to UFPI
dtypes: float64(2925)
memory usage: 64.6 MB


### Resample to weekly frequency

We start by generating weekly returns for close to 2,500 stocks without missing data for the 2008-17 period, as follows:

In [15]:
returns = (prices
           .resample('W')
           .last()
           .pct_change()
           .loc['2008': '2017']
           .dropna(axis=1)
           .sort_index(ascending=False))
returns.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 522 entries, 2017-12-31 to 2008-01-06
Freq: -1W-SUN
Columns: 2275 entries, A to UFPI
dtypes: float64(2275)
memory usage: 9.1 MB


In [16]:
returns.head().append(returns.tail())

ticker,A,AAL,AAN,AAON,AAP,AAPL,AAWW,ABAX,ABC,ABCB,...,UBSI,UCBI,UCFC,UCTT,UDR,UEC,UEIC,UFCS,UFI,UFPI
date,,,,,,,,,,,,,,,,,,,,,
2017-12-31,-0.005642,-0.010648,-0.010184,-0.001361,-0.008553,-0.033027,-0.024938,-0.001814,-0.006922,-0.019329,...,-0.025245,-0.013324,-0.018280,0.053857,0.009434,-0.016667,-0.014599,0.001098,0.018745,0.000000
2017-12-24,-0.003846,0.029965,0.090171,0.044034,-0.001490,0.006557,0.046087,0.032681,-0.007620,0.017598,...,0.007062,0.005642,-0.016913,0.002287,-0.030488,-0.021739,0.003138,-0.043487,-0.017304,0.000000
2017-12-17,0.003413,0.000784,-0.052591,-0.014006,0.003888,0.026569,0.004367,0.008396,0.074625,0.026567,...,0.011429,0.008894,-0.009424,0.037987,0.008455,0.142857,-0.007269,0.020583,-0.005551,0.000000
2017-12-10,-0.019071,0.041012,-0.005359,-0.017882,0.010375,-0.009822,-0.028014,-0.010386,0.020600,-0.054271,...,-0.057873,-0.014721,-0.021516,0.031847,-0.004844,0.210526,-0.069565,-0.026711,-0.002768,0.000000
2017-12-03,-0.009660,0.009267,0.105501,0.013947,0.112630,-0.022404,0.073838,-0.028456,0.045796,0.024717,...,0.059914,0.051991,0.031712,-0.176352,-0.011344,-0.107383,-0.053931,0.007146,-0.007418,0.000000
2008-02-03,0.038265,0.252238,0.002941,0.095182,0.097833,0.028767,0.006245,-0.078058,0.036913,0.083217,...,0.139838,0.146873,0.077453,0.084200,0.055354,0.183673,-0.011407,0.123853,0.064516,0.213845
2008-01-27,-0.013963,-0.048762,0.191310,0.071788,0.043997,-0.194286,-0.008984,-0.090807,-0.034771,0.054572,...,0.131978,0.162000,0.169014,-0.008247,0.087321,0.084071,0.018503,0.030627,0.014019,0.089204
2008-01-20,-0.065000,0.086627,-0.080541,-0.054762,-0.007176,-0.065609,0.015818,-0.019721,-0.015219,-0.044397,...,-0.078619,0.001335,-0.027397,-0.014228,0.018081,-0.244147,-0.032070,0.000686,0.019048,0.062030
2008-01-13,0.035375,-0.041902,-0.037818,-0.046538,-0.101486,-0.040878,-0.052095,0.097385,0.080137,-0.017313,...,0.016051,-0.020915,0.015905,-0.061964,0.047130,-0.091185,-0.264174,0.036234,-0.098712,-0.040750


### Create & stack 52-week sequences

We'll use 52-week sequences, which we'll create in a stacked format:

In [17]:
n = len(returns)
T = 52 # weeks
tcols = list(range(T))
tickers = returns.columns

In [18]:
data = pd.DataFrame()
for i in range(n-T-1):
    df = returns.iloc[i:i+T+1]
    date = df.index.max()    
    data = pd.concat([data, (df.reset_index(drop=True).T
                             .assign(date=date, ticker=tickers)
                             .set_index(['ticker', 'date']))])
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1066975 entries, ('A', Timestamp('2017-12-31 00:00:00')) to ('UFPI', Timestamp('2009-01-11 00:00:00'))
Data columns (total 53 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   0       1066975 non-null  float64
 1   1       1066975 non-null  float64
 2   2       1066975 non-null  float64
 3   3       1066975 non-null  float64
 4   4       1066975 non-null  float64
 5   5       1066975 non-null  float64
 6   6       1066975 non-null  float64
 7   7       1066975 non-null  float64
 8   8       1066975 non-null  float64
 9   9       1066975 non-null  float64
 10  10      1066975 non-null  float64
 11  11      1066975 non-null  float64
 12  12      1066975 non-null  float64
 13  13      1066975 non-null  float64
 14  14      1066975 non-null  float64
 15  15      1066975 non-null  float64
 16  16      1066975 non-null  float64
 17  17      1066975 non-null  float64
 18  18      1066975 non-null  flo

In [19]:
data[tcols] = (data[tcols].apply(lambda x: x.clip(lower=x.quantile(.01),
                                                  upper=x.quantile(.99))))

In [20]:
data = data.rename(columns={0: 'fwd_returns'})

In [21]:
data['label'] = (data['fwd_returns'] > 0).astype(int)

In [22]:
data.shape

(1066975, 54)

In [23]:
data.sort_index().to_hdf('data.h5', 'returns_weekly')